# ResNet-18 Model From Scratch

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def get_dataloaders(batch_size=256):
    
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4471), (0.2023, 0.1994, 0.2010))

    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4471), (0.2023, 0.1994, 0.2010))

    ])
    
    train_dataset = CIFAR10(
        root="./data", train=True, download=True, transform=train_transform,
    )

    test_dataset = CIFAR10(
        root="./data", train=False, download=True, transform=test_transform,
    )

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )

    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=True, num_workers=2
    )

    return train_loader, test_loader

In [ ]:
def get_training_protocol(model, lr=0.1, momentum=0.9, weight_decay=0.0001):
    loss = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=optimizer,
        mode='min',
        factor=0.1,
        patience=5,
        verbose=True
    )

    return loss, optimizer, scheduler

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride, downsample=None):
        super(BasicBlock, self).__init__()
        
        self.basic_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

        self.downsample = downsample
    
    def _initial_forward(self, x):
        x = self.basic_block(x)
        return x
    
    def forward(self, x):
        identity = x

        out = self._initial_forward(x)

        if self.downsample is not None:
            identity = self.downsample(x)
        
        out += identity

        out = F.relu(out)

        return out

In [ ]:
class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()

        self.initial_conv_block = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        num_blocks_per_layer = 2

        self.basic_block_1 = self._make_basic_block(64, 64, num_blocks_per_layer, stride=1)
        self.basic_block_2 = self._make_basic_block(64, 128, num_blocks_per_layer, stride=2)
        self.basic_block_3 = self._make_basic_block(128, 256, num_blocks_per_layer, stride=2)
        self.basic_block_4 = self._make_basic_block(256, 512, num_blocks_per_layer, stride=2)

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1,1))

        self.fc_layer = nn.Linear(512, num_classes)

    def _make_basic_block(self, in_channels, out_channels, num_blocks_per_layer, stride=1):

        downsample = None

        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        
        layers = []
        layers.append(BasicBlock(in_channels, out_channels, stride, downsample=downsample))

        self.in_channels = out_channels

        for _ in range(1, num_blocks_per_layer):
            layers.append(BasicBlock(self.in_channels, out_channels, stride=1, downsample=None))

        return nn.Sequential(*layers)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

    def forward(self, x):

        x = self.initial_conv_block(x)

        x = self.basic_block_1(x)
        x = self.basic_block_2(x)
        x = self.basic_block_3(x)
        x = self.basic_block_4(x)

        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        
        x = self.fc_layer(x)

        return x
